In [2]:
import sys
sys.path.insert(0, 'aitoolbox_aims_lib/src')

In [3]:
import aitoolbox.context as aictx

context = aictx.Context.get()
sources = context.get_sources()
sources.set("prompt","What is the meaning of life?")

# Setup

In [6]:
from pymilvus import connections, db, Collection
connections.connect(
  alias="default",
  user='',
  password='',
  host='localhost',
  port='19530'
)

db.using_database("embeddings")
chunks = Collection("chunks")

In [7]:
chunks.load()

In [8]:
from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/storage/projects/AIMS5.0/stuff/vector/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/storage/projects/AIMS5.0/stuff/vector/.venv/lib/python3.8/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


# Service

In [9]:
PROMPT=aictx.Context.get().get_sources().get("prompt")
print(PROMPT)

What is the meaning of life?


In [10]:
embedding = embed_model.encode(PROMPT)
print(embedding)

[-5.87179027e-02  9.47447047e-02 -6.24467582e-02  2.27122158e-02
 -5.04852012e-02  3.73861170e-03  9.86932144e-02 -1.58774480e-02
  9.85546038e-02  4.24058875e-03  3.72457206e-02 -6.09366931e-02
 -1.62903164e-02 -8.01996142e-03 -4.41915914e-03 -3.71029265e-02
 -7.82160088e-02 -3.40451561e-02 -2.46130675e-03 -2.09166221e-02
  4.27382365e-02  1.12933088e-02 -3.76934446e-02  5.37125580e-02
 -9.86241326e-02  1.11397736e-01 -5.80954459e-03  2.37018000e-02
 -2.12566709e-04  2.24462915e-02  2.84537934e-02  5.35257608e-02
  8.44105408e-02 -2.42556892e-02  2.13827975e-02  3.50872464e-02
  9.51658413e-02 -3.46690416e-02  2.08120737e-02  1.82110481e-02
 -3.94743755e-02 -2.47602146e-02 -5.44179697e-03 -3.15221469e-03
  2.88050063e-02 -2.16721045e-03  2.70437915e-02 -7.51373768e-02
  1.38010114e-01  2.03170776e-02 -6.43111020e-02  2.92909984e-02
 -4.81939092e-02 -1.33580454e-02  8.95498767e-02  1.82852857e-02
  2.34580529e-03 -2.02486757e-02  6.59064204e-02 -1.29188057e-02
 -1.80610886e-03 -5.28694

In [11]:
search_params = {
    "metric_type": "COSINE",
    "limit": 5,
    "params": {"nprobe": 10},
}

try:
    result = chunks.search([list(embedding)], "chunk_embedding", search_params, limit=3, output_fields=["chunk"])
except Exception as e:
    print(e)

In [12]:
result[0].distances

[0.4927063584327698, 0.4474804103374481, 0.4237510859966278]

In [13]:
query = "\n".join([ hit.entity.get('chunk') for hit in result[0]])
print(query)

"Never again," cried the man, "never again will we wake up in the morning  and  think Who am I? What is my purpose in life? Does it really, cosmically speaking, matter if I don't get up and  go  to work?  For today we will finally learn once and for all the plain and simple answer to all these nagging little problems  of  Life, the Universe and Everything!"
Many many millions of years ago a race of  hyperintelligent  pan- dimensional  beings  (whose  physical  manifestation in their own pan-dimensional universe is not dissimilar to our own) got so fed up  with  the  constant bickering about the meaning of life which used to interrupt  their  favourite  pastime  of  Brockian  Ultra Cricket  (a  curious  game which involved suddenly hitting people for no readily apparent reason and then running away)  that  they decided to sit down and solve their
"Yes but ..."

"But nothing! Think about it. The Meaning of  Life!  We  get  our fingers  on  that  we  can  hold every shrink in the Galaxy u

In [5]:
aictx.Context.get().get_destinations().set("resp",[ hit.entity.get('chunk') for hit in result[0]])

NameError: name 'query' is not defined